In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import math
import scipy 

import utils

git_root_path = utils.get_git_root(os.getcwd())
viz_root_path = os.path.join(git_root_path, utils.VIZ_DIR_NAME)
data_path = os.path.join(git_root_path, utils.DATA_DIR_NAME)

training_csv_path = os.path.join(data_path, utils.TRAINING_FILENAME + "." + utils.INPUT_DATA_FORMAT)
training_df = pd.read_csv(training_csv_path, sep = ";")

In [ ]:
training_df["timestamp"] = pd.to_datetime(training_df["timestamp"])
training_df["energy_balance"] = training_df["actual_pv"] - training_df["actual_consumption"]
sites = training_df["site_id"].unique()
site = 31

site_df = training_df[training_df["site_id"] == site]
periods = site_df["period_id"].unique()
period = 7
period_df = site_df[site_df["period_id"] == period]
period_df = period_df.sort_values("timestamp")
period_df = period_df.set_index("timestamp")

In [ ]:
period_df[["actual_consumption", "actual_pv", "energy_balance"]]
#num_weeks = math.ceil((max(period_df.index) - min(period_df.index)).days / 7)

In [ ]:
# import scipy 
# num_viz = 3
# viz_duration = math.ceil((max(period_df.index) - min(period_df.index)).days / num_viz)

# #num_viz = math.ceil((max(period_df.index) - min(period_df.index)).days / viz_duration)

# start_timestamp = min(period_df.index)
# print("-- period ID {}".format(period))

# fig_height = 10
# sns.set(rc={'figure.figsize':(18, fig_height)})

# fig = plt.figure()

# ymin = min(period_df.energy_balance) 
# ymin = ymin - (abs(ymin) * .1)
# ymax = max(period_df.energy_balance)
# ymax = ymax + (abs(ymax) * .1)
# # start_dt_str = min(period_df.index).strftime('%m-%d-%Y')
# # end_dt_str = max(period_df.index).strftime('%m-%d-%Y')
# # plot = plt.plot('energy_balance', data=period_df, linewidth=1, label="Energy Balance")
# # period_df['zeroes'] = 0.
# # plot = plt.fill_between(period_df.index,period_df['energy_balance'],where=period_df['energy_balance']>=period_df['zeroes'], interpolate=True, color='black')
# # plot = plt.fill_between(period_df.index, period_df['energy_balance'], where=period_df['energy_balance']<=period_df['zeroes'], interpolate=True, color='red')
# # plot = plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
# # plot = plt.title("Site ID: {site} / Period ID: {period} \n{start_dt_str} - {end_dt_str}".format(**locals()))
# for i in range(1, num_viz + 1):
#     start_dt_str = start_timestamp.strftime('%m-%d-%Y')
#     end_dt_str = (start_timestamp + timedelta(days=viz_duration)).strftime('%m-%d-%Y')
#     viz_df = period_df[(period_df.index >= start_timestamp) \
#         & (period_df.index < start_timestamp + timedelta(days=viz_duration))]

    
#     indiv_plot = fig.add_subplot(num_viz, 1, i)
#     indiv_plot.set_ylim([ymin,ymax])
#     indiv_plot = plt.plot('energy_balance', data=viz_df, linewidth=1, label="Energy Balance")
#     viz_df['zeroes'] = 0.
#     viz_df['timestamp'] = viz_df.index
#     indiv_plot = plt.fill_between(viz_df.index,viz_df['energy_balance'],where=viz_df['energy_balance']>=viz_df['zeroes'], interpolate=True, color='black')
#     indiv_plot = plt.fill_between(viz_df.index, viz_df['energy_balance'], where=viz_df['energy_balance']<=viz_df['zeroes'], interpolate=True, color='red')
#     indiv_plot = plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
#     indiv_plot = plt.title("Site ID: {site} / Period ID: {period} \n{start_dt_str} - {end_dt_str}".format(**locals()))
    
#     fig.subplots_adjust(hspace = 0.5)
#     del(indiv_plot)

#     start_timestamp = start_timestamp + timedelta(days=viz_duration)

# #fig.subplots_adjust(top = 2.0)
# viz_dir = os.path.join(viz_root_path, "energy_balance", "Site {}".format(site), "")
# filename = "Period {}.png".format(period)
# if not os.path.exists(viz_dir):
#     os.makedirs(viz_dir)
# plt.tight_layout()
# fig.savefig(os.path.join(viz_dir, filename))
# plt.close(fig)

### One plot per period

In [ ]:
import scipy 
num_viz = 3
viz_duration = math.ceil((max(period_df.index) - min(period_df.index)).days / num_viz)

#num_viz = math.ceil((max(period_df.index) - min(period_df.index)).days / viz_duration)

start_timestamp = min(period_df.index)
print("-- period ID {}".format(period))

fig_height = 10
sns.set(rc={'figure.figsize':(18, fig_height)})

fig = plt.figure()
period_df['zeroes'] = 0.
#for i in range(1, num_viz + 1):
start_dt_str = start_timestamp.strftime('%m-%d-%Y')
end_dt_str = (start_timestamp + timedelta(days=viz_duration)).strftime('%m-%d-%Y')
# viz_df = period_df[(period_df.index >= start_timestamp) \
#     & (period_df.index < start_timestamp + timedelta(days=viz_duration))]

energy_bal_ax = fig.add_subplot(1, 1, 1)
energy_bal_ax.set_ylim([ymin,ymax])
energy_bal_ax.set_ylabel("Energy (W)")
energy_bal_line = energy_bal_ax.plot('energy_balance', data=period_df, linewidth=1, label="Energy Balance", alpha=0)
energy_bal_plt = energy_bal_ax.fill_between(period_df.index,period_df['energy_balance'],where=period_df['energy_balance']>=period_df['zeroes'], interpolate=True, color='black')
energy_bal_plt = energy_bal_ax.fill_between(period_df.index, period_df['energy_balance'], where=period_df['energy_balance']<=period_df['zeroes'], interpolate=True, color='red')

price_ax = energy_bal_ax.twinx() 
price_ax.set_ylabel('Buy Price ($)', color="green")
price_ax.plot(period_df['price_buy_00'], color="green", alpha = 0.5)
price_ax.tick_params(axis='y', color="green")
plt.setp(price_ax.get_yticklabels(), color="green")

#indiv_plot = plt.plot('energy_balance', data=period_df, linewidth=1, label="Energy Balance")
#viz_df['timestamp'] = viz_df.index
#indiv_plot = plt.fill_between(period_df.index,period_df['energy_balance'],where=period_df['energy_balance']>=period_df['zeroes'], interpolate=True, color='black')
##indiv_plot = plt.fill_between(period_df.index, period_df['energy_balance'], where=period_df['energy_balance']<=period_df['zeroes'], interpolate=True, color='red')
#indiv_plot = plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
#indiv_plot = plt.title("Site ID: {site} / Period ID: {period} \n{start_dt_str} - {end_dt_str}".format(**locals()))

plt.title("Site ID: {site} / Period ID: {period} \n{start_dt_str} - {end_dt_str}".format(**locals()), fontdict={"fontsize" : 16, "fontweight": "bold"})
#     fig.subplots_adjust(hspace = 0.5)
# #     plt.show()
# #     [i.set_color("red") for i in energy_bal_ax.get_yticklabels() if float(i.get_text().replace('−', '-')) < 0]

#     start_timestamp = start_timestamp + timedelta(days=viz_duration)
# ax1 = fig.add_subplot(111)
# #ax2 = plot.twinx() 
# ymin = min(period_df.energy_balance) 
# ymin = ymin - (abs(ymin) * .1)
# ymax = max(period_df.energy_balance)
# ymax = ymax + (abs(ymax) * .1)
# start_dt_str = min(period_df.index).strftime('%m-%d-%Y')
# end_dt_str = max(period_df.index).strftime('%m-%d-%Y')
# ax1.set_ylim([ymin,ymax])
# ax1.set_ylabel("Energy (W)")
# #ax.get_xticklabels()[3].set_color("red")

# #print([i for i in plt.gca().get_yticklabels()])
# lines = ax1.plot('energy_balance', data=period_df, linewidth=1, label="Energy Balance", alpha=0)

#plot = plt.plot('energy_balance', data=period_df, linewidth=1, label="Energy Balance")
# period_df['zeroes'] = 0.
# ax2 = ax1.twinx() 
# ax2.set_ylabel('Buy Price ($)', color="blue")
# ax2.plot(period_df['price_buy_00'], color="blue")
# ax2.tick_params(axis='y', color="blue")
# plt.setp(ax2.get_yticklabels(), color="blue")

# lines = ax1.fill_between(period_df.index,period_df['energy_balance'],where=period_df['energy_balance']>=period_df['zeroes'], interpolate=True, color='black')
# lines = ax1.fill_between(period_df.index, period_df['energy_balance'], where=period_df['energy_balance']<=period_df['zeroes'], interpolate=True, color='red')
# #plot = plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
# lines = plt.title("Site ID: {site} / Period ID: {period} \n{start_dt_str} - {end_dt_str}".format(**locals()))
#[i.set_color("red") for i in ax1.get_yticklabels() if float(i.get_text().replace('−', '-')) < 0]
#print([ax1.get_yticklabels()[i].get_text() for i, lbl in enumerate(ax1.get_yticklabels())])

#plt.show()

#[i.set_color("red") for i in ax1.get_yticklabels() if float(i.get_text().replace('−', '-')) < 0]
viz_dir = os.path.join(viz_root_path, "energy_balance", "Site {}".format(site), "")
filename = "Period {}.png".format(period)
if not os.path.exists(viz_dir):
    os.makedirs(viz_dir)
plt.tight_layout()
fig.savefig(os.path.join(viz_dir, filename))
[i.set_color("red") for i in energy_bal_ax.get_yticklabels() if float(i.get_text().replace('−', '-')) < 0]
plt.show()
fig.savefig(os.path.join(viz_dir, filename))
plt.close(fig)

In [ ]:
import matplotlib.dates as mdates

num_viz = 3
viz_duration = math.ceil((max(period_df.index) - min(period_df.index)).days / num_viz)

#num_viz = math.ceil((max(period_df.index) - min(period_df.index)).days / viz_duration)

start_timestamp = min(period_df.index)
print("-- period ID {}".format(period))

fig_height = 10
sns.set(rc={'figure.figsize':(18, fig_height)})

fig = plt.figure()
plt.suptitle("Energy Balance and Price - Site ID: {site} / Period ID: {period}".format(**locals()), fontsize=20, fontweight="heavy")
period_df['zeroes'] = 0.

viz_dir = os.path.join(viz_root_path, "energy_balance", "Site {}".format(site), "")
filename = "Period {}.png".format(period)

for i in range(1, num_viz + 1):
    start_dt_str = start_timestamp.strftime('%m-%d-%Y')
    end_dt_str = (start_timestamp + timedelta(days=viz_duration)).strftime('%m-%d-%Y')
    viz_df = period_df[(period_df.index >= start_timestamp) \
         & (period_df.index < start_timestamp + timedelta(days=viz_duration))]

    energy_bal_ax = fig.add_subplot(num_viz, 1, i)
    energy_bal_ax.set_ylim([ymin,ymax])
    energy_bal_ax.set_ylabel("Energy (W)")
    energy_bal_ax.plot('energy_balance', data=viz_df, linewidth=1, label="Energy Balance", alpha=0)
    energy_bal_ax.fill_between(viz_df.index,viz_df['energy_balance'],where=viz_df['energy_balance']>=viz_df['zeroes'], interpolate=True, color='black')
    energy_bal_ax.fill_between(viz_df.index, viz_df['energy_balance'], where=viz_df['energy_balance']<=viz_df['zeroes'], interpolate=True, color='red')
    energy_bal_ax.yaxis.grid(False)
    
    
    energy_bal_ax.xaxis.set_major_locator(mdates.DayLocator(bymonthday=range(1,32), interval=1))
    energy_bal_ax.xaxis.grid(True, linestyle="--", color="gray")
    #energy_bal_ax.xaxis.set_major_locator(days)
    #energy_bal_ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))

    price_ax = energy_bal_ax.twinx() 
    price_ax.set_ylabel('Buy Price ($)', color="green")
    price_ax.plot(viz_df['price_buy_00'], color="green", alpha = 0.5)
    price_ax.tick_params(axis='y', color="green")
    price_ax.yaxis.grid(False)
    
    plt.setp(price_ax.get_yticklabels(), color="green")
    plt.title("{start_dt_str} - {end_dt_str}".format(**locals()), fontdict={"fontsize": 16})
    fig.subplots_adjust(hspace = 0.5)
    if not os.path.exists(viz_dir):
        os.makedirs(viz_dir)
    plt.tight_layout()
    fig.savefig(os.path.join(viz_dir, filename))
    [i.set_color("red") for i in energy_bal_ax.get_yticklabels() if float(i.get_text().replace('−', '-')) < 0]
    start_timestamp = start_timestamp + timedelta(days=viz_duration)

if not os.path.exists(viz_dir):
    os.makedirs(viz_dir)
plt.tight_layout()
fig.subplots_adjust(top = 0.9)

plt.show()
fig.savefig(os.path.join(viz_dir, filename))
plt.close(fig)

#indiv_plot = plt.plot('energy_balance', data=period_df, linewidth=1, label="Energy Balance")
#viz_df['timestamp'] = viz_df.index
#indiv_plot = plt.fill_between(period_df.index,period_df['energy_balance'],where=period_df['energy_balance']>=period_df['zeroes'], interpolate=True, color='black')
##indiv_plot = plt.fill_between(period_df.index, period_df['energy_balance'], where=period_df['energy_balance']<=period_df['zeroes'], interpolate=True, color='red')
#indiv_plot = plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
#indiv_plot = plt.title("Site ID: {site} / Period ID: {period} \n{start_dt_str} - {end_dt_str}".format(**locals()))


#     fig.subplots_adjust(hspace = 0.5)
# #     plt.show()
# #     [i.set_color("red") for i in energy_bal_ax.get_yticklabels() if float(i.get_text().replace('−', '-')) < 0]

#     start_timestamp = start_timestamp + timedelta(days=viz_duration)
# ax1 = fig.add_subplot(111)
# #ax2 = plot.twinx() 
# ymin = min(period_df.energy_balance) 
# ymin = ymin - (abs(ymin) * .1)
# ymax = max(period_df.energy_balance)
# ymax = ymax + (abs(ymax) * .1)
# start_dt_str = min(period_df.index).strftime('%m-%d-%Y')
# end_dt_str = max(period_df.index).strftime('%m-%d-%Y')
# ax1.set_ylim([ymin,ymax])
# ax1.set_ylabel("Energy (W)")
# #ax.get_xticklabels()[3].set_color("red")

# #print([i for i in plt.gca().get_yticklabels()])
# lines = ax1.plot('energy_balance', data=period_df, linewidth=1, label="Energy Balance", alpha=0)

#plot = plt.plot('energy_balance', data=period_df, linewidth=1, label="Energy Balance")
# period_df['zeroes'] = 0.
# ax2 = ax1.twinx() 
# ax2.set_ylabel('Buy Price ($)', color="blue")
# ax2.plot(period_df['price_buy_00'], color="blue")
# ax2.tick_params(axis='y', color="blue")
# plt.setp(ax2.get_yticklabels(), color="blue")

# lines = ax1.fill_between(period_df.index,period_df['energy_balance'],where=period_df['energy_balance']>=period_df['zeroes'], interpolate=True, color='black')
# lines = ax1.fill_between(period_df.index, period_df['energy_balance'], where=period_df['energy_balance']<=period_df['zeroes'], interpolate=True, color='red')
# #plot = plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
# lines = plt.title("Site ID: {site} / Period ID: {period} \n{start_dt_str} - {end_dt_str}".format(**locals()))
#[i.set_color("red") for i in ax1.get_yticklabels() if float(i.get_text().replace('−', '-')) < 0]
#print([ax1.get_yticklabels()[i].get_text() for i, lbl in enumerate(ax1.get_yticklabels())])

#plt.show()

#[i.set_color("red") for i in ax1.get_yticklabels() if float(i.get_text().replace('−', '-')) < 0]


In [ ]:
period_df.tail()

In [ ]:
for char in u'−200000':
    print(type(char))